In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [4]:
import pandas as pd
import numpy as np
import re
import time

In [5]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    

    image_div = soup.find('div', {'id': 'ibu_1'})
    if image_div:
        style = image_div.get('style')
        background_image_url = re.search(r'url\((.*?)\)', style).group(1)
    else:
        background_image_url = ""
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    
    for _ in range(3):
        keyword_tab_link.click()
        keyword_tab_link.click()
        time(0.5)
    try:
        keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
        keyword_list_link.click()
    except:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", "", background_image_url]

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number, background_image_url]

In [2]:
len(cafe_search_list)

23

In [6]:
cafe_search_list= [
                   "동성로", "대구 중앙로", "교동", "경북대병원역"
                    "동대구역", "대구 평화시장", "안심역",
                    "광장코아", "만평역", "대구 서구청",
                    "명덕역", "대구 앞산",
                    "경대북문", "대구 복현오거리", "동천동 카페", "칠곡네거리",
                    "범어네거리", "수성유원지", "시지", "지산",
                    "두류공원", "용산", "월배", "계명대"
                    ]

In [7]:
for cafe_name in cafe_search_list[:6]:
    try:
        df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
                                    "menu", "keyword", 'keyword_number', 'image_link'])
        # WebDriver 초기화
        driver = webdriver.Chrome() # Chrome 드라이버 사용
        driver.get(f"https://map.naver.com/p/search/{cafe_name} 카페") # 네이버 지도 접속

        # 결과 로드 대기
        time.sleep(3) # 페이지 로드를 위해 잠시 대기


        while True:
            # searchIframe으로 컨텍스트 전환 + div scroll 내리기
            driver.switch_to.frame("searchIframe")
            scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
            for _ in range(5):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
                time.sleep(1)

            # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
            cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
            cafe_size = len(cafe_list)


            for item in cafe_list:
                try:
                    place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
                    # 부모의 부모인 a 태그 찾기 및 클릭
                    parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
                    parent_a_tag.click()

                    time.sleep(3)

                    # 필요한 작업 수행 (예: 상세 정보 수집)
                    driver.switch_to.default_content()
                    driver.switch_to.frame("entryIframe")
                    cafe_row = cafe_crawling2()
                    if cafe_row:
                        df.loc[len(df)] = cafe_row
                        print(cafe_row)

                    # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전환
                    driver.switch_to.default_content()
                    driver.switch_to.frame("searchIframe")

                except Exception as e:
                    print(e)
                    pass

            next_a_link = driver.find_elements(By.CLASS_NAME, "eUTV2")[1]
            aria_disabled = next_a_link.get_attribute("aria-disabled")
            # 'aria-disabled'가 'false'이면 a 태그 클릭
            if aria_disabled == "false":
                next_a_link.click()
            else:
                break

            # 원래 프레임으로 돌아감
            driver.switch_to.default_content()
    finally:
            df.to_csv(f"./new_cafe_data/{cafe_name}.csv", encoding='utf-8')


'WebElement' object has no attribute 'double_click'
Message: stale element reference: stale element not found
  (Session info: chrome=119.0.6045.124); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7B51582B2+55298]
	(No symbol) [0x00007FF7B50C5E02]
	(No symbol) [0x00007FF7B4F805AB]
	(No symbol) [0x00007FF7B4F8E59A]
	(No symbol) [0x00007FF7B4F859E9]
	(No symbol) [0x00007FF7B4F84298]
	(No symbol) [0x00007FF7B4F86FDF]
	(No symbol) [0x00007FF7B4F870D0]
	(No symbol) [0x00007FF7B4FC13E4]
	(No symbol) [0x00007FF7B4FC18DC]
	(No symbol) [0x00007FF7B4FB819C]
	(No symbol) [0x00007FF7B4FE20EF]
	(No symbol) [0x00007FF7B4FB80CF]
	(No symbol) [0x00007FF7B4FE22C0]
	(No symbol) [0x00007FF7B4FFAAA4]
	(No symbol) [0x00007FF7B4FE1E83]
	(No symbol) [0x00007FF7B4FB670A]
	(No symbol) [0x00007FF7B4FB7964]
	GetHandleVerifier [0x00007FF7B54D0AAB+3694587]
	GetHandleVe

IndexError: list index out of range